In [2]:
from gensim.models.word2vec import Word2Vec

modeldir = '../../compass/1-year/model-sgns/'
models_sgns = []

for i in range(1946, 2024):
    year = modeldir + str(i) + '.model'
    model = Word2Vec.load(year)
    models_sgns.append(model)

In [19]:
search_word = models_sgns[-1].wv['习近平']

year = 1946
for model in models_sgns:
    repr = str(year) + ": "
    for pair in model.wv.most_similar([search_word], topn=10):
        repr += str(pair) + '\t'
    print(repr)
    year += 1

1946: ('祝词', 0.6321959495544434)	('谢忱', 0.6141595840454102)	('刘茂恩', 0.6131871938705444)	('黎玉', 0.6120835542678833)	('欢迎词', 0.6081730723381042)	('聂真', 0.6054726839065552)	('一波', 0.6001706123352051)	('来平', 0.5970344543457031)	('政委会', 0.5968754291534424)	('中国共产党中央委员会', 0.5933252573013306)	
1947: ('廖承志', 0.6492536067962646)	('张旗', 0.6415376663208008)	('波立特', 0.6386903524398804)	('开幕典礼', 0.6187020540237427)	('冯氏', 0.6133701801300049)	('李富春', 0.6087362170219421)	('延安文艺座谈会', 0.6086372137069702)	('雪峰', 0.6059626936912537)	('刘少奇', 0.6036465764045715)	('谢意', 0.5987731218338013)	
1948: ('张鼎丞', 0.7160112857818604)	('仲勋', 0.635944128036499)	('闭幕会', 0.6320939660072327)	('周文', 0.627514660358429)	('详论', 0.6183709502220154)	('冷楚', 0.6165666580200195)	('戈登', 0.6164758205413818)	('贝鲁特', 0.615566611289978)	('王培法', 0.6148638725280762)	('一般性', 0.6104544401168823)	
1949: ('黄国越', 0.6855201721191406)	('弼', 0.6680986881256104)	('谢杰', 0.6503959894180298)	('二十周年', 0.650345504283905)	('米苏', 0.6408260464668274)	('宋

In [20]:
from gensim.models.word2vec import Word2Vec

modeldir = '../../compass/1-year/model-cbow/'
models_cbow = []

for i in range(1946, 2024):
    year = modeldir + str(i) + '.model'
    model = Word2Vec.load(year)
    models_cbow.append(model)

In [24]:
search_word = models_cbow[-1].wv['国务卿']

year = 1946
for model in models_cbow:
    repr = str(year) + ": "
    for pair in model.wv.most_similar([search_word], topn=10):
        repr += str(pair) + '\t'
    print(repr)
    year += 1

1946: ('财长', 0.6920934915542603)	('总统', 0.6603286266326904)	('美前', 0.6493613719940186)	('索', 0.6190326809883118)	('朱可夫', 0.606961190700531)	('瓦尔', 0.6035975217819214)	('不久前', 0.596271276473999)	('泼', 0.5962560772895813)	('柏德逊', 0.596184492111206)	('特使', 0.5958298444747925)	
1947: ('贝纳斯', 0.6942726969718933)	('杜勒斯', 0.6667155027389526)	('国务卿', 0.6588608026504517)	('克莱顿', 0.644461989402771)	('康纳利', 0.6375260353088379)	('华莱士', 0.6342633962631226)	('美副', 0.6273282170295715)	('财长', 0.6255975961685181)	('克莱', 0.6226345300674438)	('登堡', 0.6184279918670654)	
1948: ('杜勒斯', 0.6731732487678528)	('约翰', 0.6667992472648621)	('范登堡', 0.6441597938537598)	('国务卿', 0.6311670541763306)	('宣称', 0.6310917139053345)	('特尔', 0.6261348724365234)	('杜鲁门', 0.6136705875396729)	('杜威', 0.6121715307235718)	('克莱', 0.6110726594924927)	('夫人', 0.6063442230224609)	
1949: ('总统', 0.6641810536384583)	('国务卿', 0.653583288192749)	('司法部长', 0.6460509300231934)	('大主教', 0.6454755067825317)	('赫尔', 0.642789363861084)	('贝文', 0.6420536637